# Validação Cruzada

# Índice:
   * [Descrição](#description)
      * [Objetivos](#goals)
   * [Imports](#imports)
   * [Extrção e Transformação](#extraction)
   * [Parâmtetros](#parameters)
      * [Introdução a Modelos de Árvores de Decisão](#decisiontree)
      * [Espaço de Busca](#searchspace)
      * [Log no MLFlow](#mlflow) 

## Descrição <a class="anchor" id="description"></a>

No primeiro notebook foi treinado um modelo simples de regressão linear e seu desempenho foi metrificado e a importância das características implementadas foi analisado. Tanto as métricas quanto os coeficientes das características foram calculados em um único valor. Neste notebook é implementada a técnica de validação cruzada para calcular um intervalo de confiância para cada métrica e coeficiente.

Com validação cruzada do tipo k-folds, ao invés de se medir o desempenho do modelo em com um conjunto de testes e de treino fixo, é feito um particionamento da totalidade dos dados em k conjuntos distintos. Em cada iteração um destes conjuntos é usado como teste e o restante como treino, e o modelo resultante é avaliado com o conjunto de métricas. 
No final de todas as iterações, são computados a média e o desvio padrão de cada métrica. Dessa forma, o valor final é menos dependente da escolha do conjunto de treino e de teste e a totalidade dos dados é usada para se efetuar os testes.

### Objetivos  <a class="anchor" id="goals"></a>
   * Entender a técninca de validação cruzada,
   * aplicar CV ao modelo linear,
   * desenvolver funções para a análise dos resultados.

## Imports  <a class="anchor" id="imports"></a>

In [4]:

import os
import sys
from tabnanny import verbose
import pandas as pd
import numpy as np
import yaml
from itertools import chain, combinations
import datetime
import tempfile

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import permutation_test_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.utils import shuffle

import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from mlflow.utils.mlflow_tags import MLFLOW_PARENT_RUN_ID


## Extração e Transformação <a class="anchor" id="extraction"></a>

In [2]:
def read_data(url):
    return pd.read_csv(os.path.abspath(url))

In [3]:
df = read_data("../extracao/data.csv")

In [4]:
def getXy(df):
    r_state = 15
    df = shuffle(df, random_state=r_state)
    X = df.iloc[:,:-1]
    y = df.iloc[:,-1]
    return X, y
r_state = 12
X, y = getXy(df)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=r_state
)


## Treinamento <a class="anchor" id="training"></a>

Aqui é definida uma estratégia de validação cruzada, que é feita com o método k-folds utilizando 8 folds. As métricas são as definidas no último notebook: raíz do erro quadrático médio, erro absoluto médio e coeficiente de determinação.

In [5]:
def cross_validation(model, _X, _y, _cv):
    _scoring = ["r2", "neg_mean_absolute_error", "neg_root_mean_squared_error"]
    results = cross_validate(estimator=model, X=_X, y=_y, cv=_cv, scoring=_scoring)
    return results


In [5]:
model= make_pipeline(StandardScaler(), LinearRegression())

In [7]:
results = cross_validation(model,X, y, 8)

In [8]:
results

{'fit_time': array([0.32531357, 0.00421739, 0.00339985, 0.00335693, 0.00358605,
        0.00336933, 0.00335431, 0.00368881]),
 'score_time': array([0.01016212, 0.00247717, 0.00189805, 0.00192094, 0.00193977,
        0.00192165, 0.00201154, 0.00194788]),
 'test_r2': array([0.17411628, 0.17459729, 0.19439588, 0.19286014, 0.1996818 ,
        0.18111883, 0.19396096, 0.15352567]),
 'test_neg_mean_absolute_error': array([-10.75934087, -10.57516785, -10.75875503, -10.68354239,
        -10.329878  , -10.68127985, -10.41664408, -10.85006594]),
 'test_neg_root_mean_squared_error': array([-13.44744964, -13.24817279, -13.56784077, -13.62223613,
        -13.23660009, -13.59349831, -13.02502461, -13.75203714])}

## Validação <a class="anchor" id="validation"></a>

Para a validação das métricas foi escrita uma função que produz um intervalo de confiância de ... para cada métrica.

In [18]:
def eval_metrics_cv(cv):
    rmse = cv["test_neg_root_mean_squared_error"].mean()
    rmseStd = cv["test_neg_root_mean_squared_error"].std()
    mae = cv["test_neg_mean_absolute_error"].mean()
    maeStd = cv["test_neg_mean_absolute_error"].std()
    r2 = cv["test_r2"].mean()
    r2Std = cv["test_r2"].std()
    return {
            "rmse":[rmse, rmseStd],
            "mae":[mae, maeStd], 
            "r2":[r2, r2Std]
           }


In [19]:
scores = eval_metrics_cv(results)

In [20]:
def print_metrics_cv(scores):
    for metric, score in scores.items():
        mean = score[0]
        std = score[1]
        print (f"{metric}: {mean:.2f} +/- {2*std:.2f}")

In [21]:
print_metrics_cv(scores)

rmse: -13.44 +/- 0.46
mae: -10.63 +/- 0.34
r2: 0.18 +/- 0.03


## Log no MLFlow <a class="anchor" id="mlflow"></a>

In [13]:
def connectMLFlow(MLFlowAddr):
    client = MlflowClient(tracking_uri=MLFlowAddr)
    mlflow.set_tracking_uri(MLFlowAddr)
    return client

In [14]:
client = connectMLFlow("http://172.27.0.1:5000")

In [15]:
experiment_name = "Simple Model"
try:
    experiment_id = client.create_experiment(experiment_name)
except:
    experiment_id = client.get_experiment_by_name(experiment_name).experiment_id
    
experiment = mlflow.set_experiment(experiment_name)

In [24]:
run_name = "LinearRegression_default_params_CV"

with mlflow.start_run(run_name=run_name) as run:
    mlflow.log_metric("r2", scores["r2"][0])
    mlflow.log_metric("r2_std", scores["r2"][1])
    mlflow.log_metric("rmse", abs(scores["rmse"][0]))
    mlflow.log_metric("rmse_std", scores["rmse"][1])
    mlflow.log_metric("mae", abs(scores["mae"][0]))
    mlflow.log_metric("mae_std", scores["mae"][1])
    mlflow.end_run()

In [25]:
pscore = permutation_test_score(LinearRegression(), X, y)